In [17]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
import pickle
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]
_CSV_DATA_FILENAME: str = os.environ["_csv_data_filename"]



import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [18]:
# import all your modules here
import time
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

In [19]:
#blob_name = r"data/data_weekly_report.pkl"
#downloader.upload_pkl_file(bancolombia_pkl,blob_name, blob_container)

In [20]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY
#study = ['2025-01-01', '2025-02-01']

In [21]:
print(study)
print(baseline)

['2025-03-03', '2025-03-10']
['2023-01-01', '2025-03-03']


In [22]:
df_devices = ubi.get_available_devices_v2(label='bancolombia-f2', level='group', page_size=1000)
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [23]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [24]:
#leer el archivo data_weekly_report.pkl y guardarlo en un dataframe llamado df1
df1 = pd.read_pickle(r'C:\Digitalización\Bancolombia\CB_informe_Fase_2\data\data_weekly_report.pkl')


In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26523102 entries, 2024-10-07 00:00:00-05:00 to 2025-03-03 00:03:00-05:00
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   value        float64
 1   variable     object 
 2   device       object 
 3   device_name  object 
dtypes: float64(1), object(3)
memory usage: 1011.8+ MB


In [26]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': study[0], 'end': study[1]}
df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Study:{study[0]}, Study:{study[1]}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]
    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )
    if response.status_code == 204 or response.status_code >= 500:
        print(f"Empty response for chunk {idx}")
        time.sleep(10)
        response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST,)
    current_idx = idx_end+1
    if (current_idx > n_vars):
        current_idx = n_vars
    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")
    print(f"Response status code: {response.status_code}")
    if (response.status_code != 204) and  (len(response.json()['results']) >0 ):
        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")
df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)

Making request for the following interval: Study:2025-03-03, Study:2025-03-10
Progress: 3.0%
Response status code: 200
Progress: 5.7%
Response status code: 200
Progress: 8.5%
Response status code: 200
Progress: 11.2%
Response status code: 200
Progress: 13.9%
Response status code: 200
Progress: 16.7%
Response status code: 200
Progress: 19.4%
Response status code: 200
Progress: 22.1%
Response status code: 200
Progress: 24.9%
Response status code: 200
Progress: 27.6%
Response status code: 200
Progress: 30.3%
Response status code: 200
Progress: 33.1%
Response status code: 200
Progress: 35.8%
Response status code: 200
Progress: 38.5%
Response status code: 200
Progress: 41.3%
Response status code: 200
Progress: 44.0%
Response status code: 200
Progress: 46.7%
Response status code: 200
Progress: 49.5%
Response status code: 200
Progress: 52.2%
Response status code: 200
Progress: 54.9%
Response status code: 200
Progress: 57.7%
Response status code: 200
Progress: 60.4%
Response status code: 200
P

In [27]:
df.sort_index()

,value,variable,device,device_name
datetime,,,,
2025-03-03 00:00:00-05:00,5.60,front-potencia-activa,bc80t,BC 689 - Metropolis
2025-03-03 00:00:00-05:00,20.00,tr,bc90t,BC 388 - CC Hayuelos
2025-03-03 00:00:00-05:00,0.00,ilu-consumo-activa,bc90t,BC 388 - CC Hayuelos
2025-03-03 00:00:00-05:00,10.00,tr,bc-673---calle-80,BC 673 - Calle 80
2025-03-03 00:00:00-05:00,4.35,aa-potencia-activa,bc---metropolitan,BC - Metropolitan
...,...,...,...,...
2025-03-10 00:00:00-05:00,10.40,front-potencia-activa,bc-554---mall-plaza-buenavista,BC 554 - Mall Plaza Buenavista
2025-03-10 00:00:00-05:00,2.00,front-consumo-activa,bc-772---caicedonia,BC 772 - Caicedonia
2025-03-10 00:00:00-05:00,1.80,front-potencia-activa,bc-772---caicedonia,BC 772 - Caicedonia


In [28]:
df1.sort_index()

,value,variable,device,device_name
datetime,,,,
2023-12-01 00:00:00-05:00,7.000000,front-consumo-activa,bc90t,BC 388 - CC Hayuelos
2023-12-01 00:00:00-05:00,0.730000,aa-consumo-activa,bc80t,BC 689 - Metropolis
2023-12-01 00:00:00-05:00,2.000000,front-consumo-activa,bc80t,BC 689 - Metropolis
2023-12-01 00:00:00-05:00,0.005839,kw-tr,bc80t,BC 689 - Metropolis
2023-12-01 00:00:00-05:00,2.250000,aa-potencia-activa,bc88t,BC 332 - Zipaquira
...,...,...,...,...
2025-03-10 00:00:00-05:00,2.720000,front-consumo-activa,bc-478---mix-via-40,BC 478 - Mix Vía 40
2025-03-10 00:00:00-05:00,0.720000,aa-consumo-activa,bc-478---mix-via-40,BC 478 - Mix Vía 40
2025-03-10 00:00:00-05:00,0.460000,ilu-consumo-activa,bc-478---mix-via-40,BC 478 - Mix Vía 40


In [29]:
#Unir el dataframe dfprogress: 30.5%Response status code: 200Progress: 34.8%Response status code: 200Progress: 39.1%Response status code: 200Progress: 43.3%Response status code: 200 y df1 en uno que se llame df_full / esto es para mas adelante
df = pd.concat([df1, df], axis=0, ignore_index=False)

# para quitar el indixe de "datatime" y volverlo columna 
df.reset_index(inplace=True)

# Quitar duplicados 
df = df.drop_duplicates()

# Establecer la columna "datetime" como el índice
df.set_index('datetime', inplace=True)


In [30]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)

In [31]:
df.sort_index()

,value,variable,device,device_name
datetime,,,,
2023-12-01 00:00:00-05:00,8.00,front-potencia-activa,bc84t,BC 367 - Granada Meta
2023-12-01 00:00:00-05:00,180.18,area,bc89t,BC 384 - Anapoima
2023-12-01 00:00:00-05:00,2.10,aa-consumo-activa,bc88t,BC 332 - Zipaquira
2023-12-01 00:00:00-05:00,2.25,aa-potencia-activa,bc88t,BC 332 - Zipaquira
2023-12-01 00:00:00-05:00,2.60,front-potencia-activa,bc85t,BC 802 - Puerto Lopez
...,...,...,...,...
2025-03-10 00:00:00-05:00,22.50,tr,bc-115---circunvalar-pereira,BC 115 - Circunvalar Pereira
2025-03-10 00:00:00-05:00,440.20,area,bc-832---cucuta,BC 832 - San Francisco de Paula
2025-03-10 00:00:00-05:00,0.58,ilu-consumo-activa,bc-81---avenida-kennedy,BC 81 - Avenida Kennedy


In [32]:
df.index[(df.index.month == 1)&(df.index.year == 2025)].unique()

DatetimeIndex(['2025-01-07 00:00:00-05:00', '2025-01-06 23:57:00-05:00',
               '2025-01-06 23:54:00-05:00', '2025-01-06 23:51:00-05:00',
               '2025-01-06 23:48:00-05:00', '2025-01-06 23:45:00-05:00',
               '2025-01-06 23:42:00-05:00', '2025-01-06 23:39:00-05:00',
               '2025-01-06 23:36:00-05:00', '2025-01-06 23:33:00-05:00',
               ...
               '2025-01-27 01:05:00-05:00', '2025-01-27 00:55:00-05:00',
               '2025-01-27 00:50:00-05:00', '2025-01-27 00:40:00-05:00',
               '2025-01-27 00:35:00-05:00', '2025-01-27 00:25:00-05:00',
               '2025-01-27 00:20:00-05:00', '2025-01-27 00:10:00-05:00',
               '2025-01-27 00:05:00-05:00', '2025-01-28 16:57:01-05:00'],
              dtype='datetime64[ns, America/Bogota]', name='datetime', length=24491, freq=None)